In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

coll = client.collections.create(
    "<COLLECTION_NAME>",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(),
    properties=[
        Property(name="<PROP1>", data_type=DataType.TEXT),
        Property(name="<PROP2>", data_type=DataType.TEXT),
    ],
    multi_tenancy_config=Configure.multi_tenancy(True)
)

In [ ]:
from weaviate.classes.tenants import Tenant

# Add Tenant
coll = client.collections.get("<COLLECTION_NAME>")
coll.tenants.create([Tenant(name='<TENANT1>'), Tenant(name='TENANT2'), Tenant(name='TENANT3')])


In [ ]:
# How to Insert data in the tenants
# Define objects to insert into each tenant, with prop1 and prop2
data_objects = {
    "Tenant1": [
        {"prop1": "This is content for Tenant1 1", "prop2": "Content Tenant1 1"},
        {"prop1": "This is content for Tenant1 2", "prop2": "Content Tenant1 2"},
        {"prop1": "This is content for Tenant1 3", "prop2": "Content Tenant1 3"},
        {"prop1": "This is content for Tenant1 4", "prop2": "Content Tenant1 4"},
        {"prop1": "This is content for Tenant1 5", "prop2": "Content Tenant1 5"}
    ],
    "Tenant2": [
        {"prop1": "This is content for Tenant2 1", "prop2": "Content Tenant2 1"},
        {"prop1": "This is content for Tenant2 2", "prop2": "Content Tenant2 2"},
        {"prop1": "This is content for Tenant2 3", "prop2": "Content Tenant2 3"},
        {"prop1": "This is content for Tenant2 4", "prop2": "Content Tenant2 4"},
        {"prop1": "This is content for Tenant2 5", "prop2": "Content Tenant2 5"}
    ],
    "Tenant3": [
        {"prop1": "This is content for Tenant3 1", "prop2": "Content Tenant3 1"},
        {"prop1": "This is content for Tenant3 2", "prop2": "Content Tenant3 2"},
        {"prop1": "This is content for Tenant3 3", "prop2": "Content Tenant3 3"},
        {"prop1": "This is content for Tenant3 4", "prop2": "Content Tenant3 4"},
        {"prop1": "This is content for Tenant3 5", "prop2": "Content Tenant3 5"}
    ]
}

# Iterate through the tenants and insert data
for tenant_name, tenant_obj in tenants.items():
    tenant_collection = coll.with_tenant(tenant_name)  # Tenant-specific collection

# Iterate through the objects for each tenant and insert into properties
for data in data_objects[tenant_name]:
    tenant_collection.data.insert(
    properties={
        "prop1": data["prop1"],
        "prop2": data["prop2"]
        }
    )

In [ ]:
# How to update the tenants status
from weaviate.classes.tenants import Tenant, TenantActivityStatus

col = client.collections.get("<COLLECTION_NAME>")
col.tenants.update(tenants=[
    Tenant(
        name="Tenant1",
        activity_status=TenantActivityStatus.INACTIVE
    ),
    Tenant(
        name="Tenant2",
        activity_status=TenantActivityStatus.INACTIVE
    ),
    Tenant(
        name="Tenant3",
        activity_status=TenantActivityStatus.INACTIVE
    )
])

In [ ]:
# Retrieve Tenant Activity Status
import pandas as pd

col = client.collections.get("<COLLECTION_NAME>")
tenants = col.tenants.get()

# Extract tenant data into a list of dictionaries
tenant_data = []
for tenant_id, tenant in tenants.items():
    tenant_data.append({
        'Tenant ID': tenant_id,
        'Name': tenant.name,
        'Activity Status Internal': tenant.activityStatusInternal.name,
        'Activity Status': tenant.activityStatus.name
    })

# Convert the data to a pandas DataFrame
df = pd.DataFrame(tenant_data)
# Print the formatted table
print(df)


In [ ]:
# 
try:
    # Get all collections
    collections = client.collections.list_all()
    
    if collections:
        print("Collections in Weaviate:")
        for collection_name in collections.keys():
            try:
                collection = client.collections.get(collection_name)
                # Attempt to get tenants, if it fails with FAILED_PRECONDITION, MT is not enabled
                try:
                    tenants = collection.tenants.get()
                    print(f"- {collection_name}: MT is enabled")
                except Exception as tenant_error:
                    if "multi-tenancy is not enabled" in str(tenant_error):
                        print(f"- {collection_name}: MT is not enabled")
                    else:
                        print(f"Error checking multi-tenancy for '{collection_name}': {tenant_error}")
            except Exception as e:
                print(f"Error retrieving collection '{collection_name}': {e}")
    else:
        print("No collections found.")

except Exception as e:
    print(f"Error retrieving collections: {e}")